## init

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
from ase.io import read, write
from ase.visualize import view

from autoadsorbate import Surface

import sys
sys.path.insert(0, '/home/djrm/git/hetcattoolbox')
# sys.path.insert(0, '/home/djrm/git/autoadsorbate')

from hetcattoolbox.hetcattoolbox import MPResterSimple
from hetcattoolbox.utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
# import inspect
# inspect.getabsfile(Surface)

In [8]:
api_key = "0FZIs3w6bTqjjZdjdrc4G0wCQpKj15Ai"
mprs = MPResterSimple(api_key=api_key)
df = mprs.query_by_comp('Cu')
# df = drop_excess_columns(df)

Retrieving SummaryDoc documents:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# #make mappings for some string info to keep in the group by dataframe
# mp_id_map = {}
# for i, mpid in enumerate(df.material_id.values):
#     mp_id_map[i] = mpid
# df['mpid'] = [k for k in mp_id_map.keys()]

# formula_anonymous_map = {}
# for i, fa in enumerate(df.formula_anonymous.values):
#     formula_anonymous_map[i] = fa
# df['formula_anonymous_id'] = [k for k in formula_anonymous_map.keys()]

# #group df
# select_df= df.groupby(['cs', 'formula_anonymous'], as_index=False).min('energy_above_hull')

# #map back string data
# select_ids = [str(mp_id_map[mpid]) for mpid in select_df.mpid.values]
# select_df['material_id'] = select_ids

# formula_anonymous = [str(formula_anonymous_map[idx]) for idx in select_df.formula_anonymous_id.values]
# select_df['formula_anonymous'] = formula_anonymous


# select_ids

In [3]:
# df

In [4]:
# select_df = df[df.density > 5]
# select_df = df[df.energy_above_hull < 0.05]
# select_df

In [5]:
# sns.histplot(select_df.energy_above_hull.values, bins=100)

In [6]:
# _df = select_df[(select_df.nsites < 50) & (select_df.energy_above_hull<0.001)]
# bulk_trj = []
# for mpid in _df.material_id.values:
#         atoms = mprs.get_atoms_by_material_id(mpid) 
#         atoms.info['mpid'] = str(mpid)
#         bulk_trj.append(atoms)

In [9]:
manual_select= {
        'Cu': 'mp-30',
        'CuO': 'mp-1692',
        'PdO': 'mp-1336',
        'PdS': 'mp-20250',
        'CuS': 'mp-760381'
}

bulk_trj= []
for mpid in manual_select.values():
        atoms = mprs.get_atoms_by_material_id(mpid) 
        atoms.info['mpid'] = str(mpid)
        atoms.info['material_formula'] = atoms.get_chemical_formula(empirical=True)
        bulk_trj.append(atoms)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
view(bulk_trj)

<Popen: returncode: None args: ['/home/djrm/miniconda3/envs/aads/bin/python'...>

In [100]:
construct_recepie= [
    {
        'mpid': 'mp-30',
        'mi': '1#1#1',
        'iterm': 0,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 7,
        'min_b' : 7,
     },
     {
        'mpid': 'mp-30',
        'mi': '2#1#1',
        'iterm': 0,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 7,
        'min_b' : 6,
     },
     {
        'mpid': 'mp-30',
        'mi': '1#0#0',
        'iterm': 0,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 7,
        'min_b' : 7,
     },
    {
        'mpid': 'mp-1692',
        'mi': '1#1#0',
        'iterm': 2,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 10,
        'min_b' : 10,
     },
    {
        'mpid': 'mp-1336',
        'mi': '1#0#0',
        'iterm': 0,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 10,
        'min_b' : 10,
     },
    {
        'mpid': 'mp-20250',
        'mi': '0#0#1',
        'iterm': 0,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 10,
        'min_b' : 10,
     },
    {
        'mpid': 'mp-760381',
        'mi': '1#1#1',
        'iterm': 0,
        'vacuum': 10,
        'layers' : 7,
        'min_a' : 10,
        'min_b' : 10,
     }
      
]

In [ ]:
slab_trj=[]
for r in construct_recepie:
    mpid = r['mpid']
    atoms = mprs.get_atoms_by_material_id(mpid) 
    r['mpid'] = str(mpid)
    r['material_formula'] = atoms.get_chemical_formula(empirical=True)
    
    mi = str_to_mi(r['mi'])
    print(mi)

    out_atoms = get_scalled_slabs(
        atoms,
        layers = r['layers'],
        vacuum = r['vacuum'],
        mi = mi,
        min_a = r['min_a'],
        min_b = r['min_b'],
    )[r['iterm']]
    r['mi'] = mi_to_str(mi)
    out_atoms.info['slab_info'] = r
    
    pop_k = [k for k in out_atoms.info.keys() if k not in ['slab_info']]
    for k in pop_k:
        out_atoms.info.pop(k)
    

    slab_trj.append(out_atoms)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[1, 1, 1]
Using provided mi for generation: [1, 1, 1]


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[2, 1, 1]
Using provided mi for generation: [2, 1, 1]


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

[1, 0, 0]
Using provided mi for generation: [1, 0, 0]


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[0, 0, 1]
Using provided mi for generation: [0, 0, 1]


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[1, 1, 1]
Using provided mi for generation: [1, 1, 1]


In [136]:
# atoms.get_chemical_formula?

In [ ]:
# view(slab_trj)

In [139]:
# slab_trj[0].info['slab_info']

In [117]:
# trj = []

# for bulk in bulk_trj:
#         trj += get_scalled_slabs(
#                 bulk,
#                 layers = 7,
#                 vacuum = 5,
#                 max_mi = 3,
#                 min_a = 6,
#                 min_b = 6,
#         )

In [118]:
# x = trj[330]
# x = trj[842]
# # x = trj[64]
# # x = trj[263]

# print(x.get_chemical_formula(empirical=True))
# view(x)
# x.info



In [141]:
# write('./trj.xyz', trj)

In [142]:
surfaces = []
for atoms in slab_trj:
    print(atoms.info)
    s = Surface(atoms, precision=0.5)
    s.sym_reduce()
    surfaces.append(s)

{'slab_info': {'mpid': 'mp-30', 'mi': '1#1#1', 'iterm': 0, 'vacuum': 10, 'layers': 7, 'min_a': 7, 'min_b': 7, 'material_formula': 'Cu'}}
{'slab_info': {'mpid': 'mp-30', 'mi': '2#1#1', 'iterm': 0, 'vacuum': 10, 'layers': 7, 'min_a': 7, 'min_b': 6, 'material_formula': 'Cu'}}
{'slab_info': {'mpid': 'mp-30', 'mi': '1#0#0', 'iterm': 0, 'vacuum': 10, 'layers': 7, 'min_a': 7, 'min_b': 7, 'material_formula': 'Cu'}}
{'slab_info': {'mpid': 'mp-1692', 'mi': '1#1#0', 'iterm': 2, 'vacuum': 10, 'layers': 7, 'min_a': 10, 'min_b': 10, 'material_formula': 'CuO'}}
{'slab_info': {'mpid': 'mp-1336', 'mi': '1#0#0', 'iterm': 0, 'vacuum': 10, 'layers': 7, 'min_a': 10, 'min_b': 10, 'material_formula': 'OPd'}}
{'slab_info': {'mpid': 'mp-20250', 'mi': '0#0#1', 'iterm': 0, 'vacuum': 10, 'layers': 7, 'min_a': 10, 'min_b': 10, 'material_formula': 'PdS'}}
{'slab_info': {'mpid': 'mp-760381', 'mi': '1#1#1', 'iterm': 0, 'vacuum': 10, 'layers': 7, 'min_a': 10, 'min_b': 10, 'material_formula': 'CuS'}}


In [153]:
sites_traj = []
for s in surfaces:
    atoms = s.atoms
    for i in s.site_df.index.values:
        site_info = s.site_df.loc[i].to_dict()
        atoms.info['site_info'] =site_info
        sites_traj.append(atoms)
write('./sites_traj.xyz', sites_traj)


/home/djrm/miniconda3/envs/aads/lib/python3.12/site-packages/ase/io/extxyz.py:311: UserWarning: Skipping unhashable information site_info
  warnings.warn('Skipping unhashable information '


In [154]:
view(sites_traj)    

<Popen: returncode: None args: ['/home/djrm/miniconda3/envs/aads/bin/python'...>

In [ ]:
xx = pd.DataFrame({})
xx.sort

""


In [148]:
# for s in surfaces:
#     s.site_df.to_csv(f'{s.atoms.info['slab_info']['mpid']}_cache_site_df.csv')

In [140]:
# from dscribe.descriptors import SOAP

# species = get_all_species_in_traj(trj)
# r_cut = 6.0
# n_max = 8
# l_max = 6

# # Setting up the SOAP descriptor
# soap = SOAP(
#     species=species,
#     periodic=True,
#     r_cut=r_cut,
#     n_max=n_max,
#     l_max=l_max,
# )

# view_trj= []
# site_pos=[]
# soaps = []

# for s in surfaces:
#     for i in s.site_df.index.values:
#         pos = s.site_df.coordinates.loc[i]
#         view_trj.append(s.view_site(i, return_atoms=True))
#         site_pos.append(pos)
#         soaps.append(soap.create(s.atoms, centers=[pos]))